## Пространства имен

### Основы областей видимости (scopes) в Python
```markdown
- **Имена** в программе существуют в пространстве имён (namespace).  
- **Область видимости (scope)** определяется местом присваивания имени в коде.  
- Все имена создаются в момент присваивания (объявления заранее нет).  
- Функции добавляют новый уровень пространства имён, чтобы избежать конфликтов переменных.  
- Правила:  
  - имена, присвоенные внутри `def` или `lambda`, локальны;  
  - имена, присвоенные в окружающей функции, — **nonlocal**;  
  - имена, присвоенные вне всех функций, — глобальные.  
- Это называется **лексическая область видимости**: она зависит только от расположения кода, а не от того, кто кого вызывает.  
- Переменные с одинаковыми именами внутри и снаружи функции — разные переменные. 

In [1]:
X = 99  # глобальная переменная

def func():
    X = 88  # локальная переменная, своя копия
    print("Внутри функции X =", X)

func()
print("Снаружи функции X =", X)

Внутри функции X = 88
Снаружи функции X = 99


### Обзор областей видимости в Python
```markdown
- До появления функций имена создавались только:  
  - в модуле (глобальная область видимости для файла);  
  - в пространстве имён встроенных функций (`open`, `zip` и др.).  
- Функции добавляют вложенные области видимости, чтобы их имена не конфликтовали с внешними.  

## Тонкости:
```markdown
1. В интерактивной оболочке (`REPL`) всё работает как в модуле: переменные на верхнем уровне считаются глобальными.  
2. Любое присваивание делает имя локальным или глобальным в зависимости от места. Это относится к `=`, `:=`, аргументам функций, `import` и т. д.  
3. Изменение объекта **не меняет область видимости имени**. На 
   - `L = X` внутри функции создаёт локальную переменную `L`.  
   - `L.append(X)` меняет объект, на который указывает глобальная переменная `L`, и не требует объявления `global`.  

## Ключевые правила:
```markdown
- **Глобальная область (global scope)** — верхний уровень файла (модуля). Переменные, созданные здесь, становятся атрибутами модуля.  
- **Глобальная область ограничена файлом.** Нет «универсального» глобального пространства для всех модулей, кроме встроенных функций. Для доступа к именам из других файлов используется `import`.  
- **Локальная область (local scope)** — создаётся внутри `def` или `lambda`. Все имена, присвоенные внутри функции, по умолчанию локальные.  
- Чтобы изменить переменную уровня модуля из функции → используем `global`.  
- Чтобы изменить переменную внешней функции из вложенной → используем `nonlocal`.  
- Имена, на которые нет присваивания внутри функции, ищутся во внешних областях: enclosing function → global → built-ins.  
- **Каждый вызов функции** создаёт новое локальное пространство имён. Это важно для рекурсии и замыканий: каждый вызов получает собственный набор локальных переменных.  

### Правило LEGB для разрешения имён
```markdown
- Внутри функции (`def` или `lambda`) присваивания по умолчанию создают **локальные имена**.  
- Поиск имён при обращении к ним без точки (`name`) идёт в порядке **LEGB**:  
  1. **L (Local)** — локальная область текущей функции.  
  2. **E (Enclosing)** — локальные области всех внешних функций (если есть).  
  3. **G (Global)** — глобальная область текущего модуля.  
  4. **B (Built-in)** — встроенные имена Python (`len`, `open`, `range` и др.).  
- Поиск останавливается на первом найденном совпадении.  
- Чтобы изменить переменные во внешних областях, используются `global` и `nonlocal`.  
- На верхнем уровне файла локальная и глобальная области совпадают.  
- Важно: это относится только к простым именам. Для выражений вида `object.name` действуют другие правила (связанные с объектами и наследованием).  


In [2]:
# Глобальная переменная
X = 99  

def outer():
    X = 88  # Enclosing переменная

    def inner():
        X = 77  # Local переменная
        print("Local:", X)  # Берётся из inner

    inner()
    print("Enclosing:", X)  # Берётся из outer

outer()
print("Global:", X)  # Берётся из модуля
print("Built-in:", len([1, 2, 3]))  # Пример встроенной функции

Local: 77
Enclosing: 88
Global: 99
Built-in: 3


### Дополнительные области видимости в Python
```markdown
- Помимо стандартных областей LEGB, существуют ещё три особых случая:  
  1. **Переменные в comprehensions** (`[x for x in I]`): имя `x` локально только для comprehension и не конфликтует с внешними переменными. В обычных циклах `for` переменная цикла остаётся доступной снаружи.  
  2. **Переменные в except-блоках** (`except E as X`): имя `X` локально только в блоке `except` и удаляется при выходе из него.  
  3. **Присваивания внутри lambda с :=**: создают локальные переменные внутри выражения `lambda`.  
- Все эти контексты расширяют правило LEGB, но не меняют его принципиально.  
- **class** также создаёт собственное локальное пространство имён:  
  - имена внутри блока класса становятся атрибутами класса после завершения определения;  
  - область видимости класса не учитывается при поиске имён (Python ищет enclosing functions, но не enclosing classes).  
- Важно: классы не создают область на каждый вызов (в отличие от функций). При вызове класса создаются объекты, которые получают собственные атрибуты.  


In [3]:
# Comprehension: переменная x локальна
squares = [x*x for x in range(3)]
try:
    print(x)  # Ошибка: x не определён
except NameError:
    print("x не доступен вне comprehension")

x не доступен вне comprehension


In [4]:
# Обычный for: переменная сохраняется
for y in range(3):
    pass
print("Переменная из for:", y)

Переменная из for: 2


In [5]:
# Переменная в except-блоке
try:
    1 / 0
except ZeroDivisionError as e:
    print("Ошибка внутри except:", e)
try:
    print(e)  # Ошибка: e удалён
except NameError:
    print("e не доступен вне except")

Ошибка внутри except: division by zero
e не доступен вне except


In [6]:
# Пример класса
class MyClass:
    attr = 42
    def method(self):
        return self.attr

obj = MyClass()
print("Атрибут класса:", obj.attr)

Атрибут класса: 42


### Встроенная область видимости
```markdown
- Встроенная область (`Built-in scope`) реализована модулем **`builtins`**.  
- Все встроенные функции и исключения (около 150 имён в Python 3.12) доступны автоматически по правилу **LEGB** (уровень `B`).  
- Импорт `builtins` позволяет просматривать список встроенных имён через `dir(builtins)`.  
- Локальные и глобальные переменные могут **затенять** (shadow) встроенные имена.  
- Затенение может быть полезным, но чаще это источник ошибок.  
- Специальные имена `None`, `True`, `False` зарезервированы и не могут быть изменены.  
- Чтобы восстановить встроенное имя после случайного затенения, можно использовать `del name` или перезапустить сессию.  


In [10]:
import builtins

# Встроенная функция доступна по двум маршрутам
print(zip is builtins.zip)  # True

# Опасность затенения
def hider():
    open = "text"  # локальная переменная скрывает встроенный open
    open("data.txt")  

hider()

True


TypeError: 'str' object is not callable

In [12]:
# Затенение на глобальном уровне
open = 99
print("Затенённый open:", open)

# Удалим имя, чтобы вернуть встроенную функцию
del open
print("Восстановленный open:", open("data.txt"))  

Затенённый open: 99
Восстановленный open: <_io.TextIOWrapper name='data.txt' mode='r' encoding='UTF-8'>


In [13]:
# Локальное имя скрывает глобальное
X = 88
def func():
    X = 99
    print("Внутри функции:", X)

func()
print("Снаружи функции:", X)

Внутри функции: 99
Снаружи функции: 88


### Проектирование программ: минимизация глобальных переменных
```markdown
- Функции должны общаться через **аргументы и возвращаемые значения**, а не через глобальные переменные.  
- По умолчанию переменные в функциях локальные → изменить глобальные можно только явно (`global`).  
- Минусы глобальных переменных:  
  - значение зависит от порядка вызова функций;  
  - код труднее понимать и сопровождать;  
  - сильная зависимость функций друг от друга.  
- Глобальные переменные можно использовать для хранения состояния, но это крайний случай.  
- В многопоточности глобальные переменные выступают как разделяемая память, но требуют аккуратности.  


In [14]:
# Плохой дизайн: функции зависят от общей переменной X
X = 99

def func1():
    global X
    X = 88

def func2():
    global X
    X = 77

func1()
print(X)  # 88
func2()
print(X)  # 77 (значение зависит от порядка вызова)

88
77


### Другие способы доступа к глобальным переменным
```markdown
- Глобальные переменные = атрибуты модуля.  
- Способы изменения глобальной переменной:  
  1. С помощью `global` внутри функции (рекомендуемый) 
  2. Через импорт самого модуля (`import thismod`) и изменение его атрибутов.  
  3. Через словарь `sys.modules`, где хранятся все загруженные модули.  
- Все эти методы работают, но обычно используют именно `global`, так как он проще и явно показывает намерения.  


In [18]:
# thismod.py

var = 99  # глобальная переменная = атрибут модуля

def local():
    var = 0  # локальное изменение, не влияет на глобальную

def glob1():
    global var
    var += 1  # явное изменение через global

def glob2():
    import thismod
    thismod.var += 1  # изменение через импорт модуля

def glob3():
    import sys
    thismod = sys.modules['thismod']
    thismod.var += 1  # изменение через sys.modules
